In [1]:
import os
import pandas as pd
import requests
from sklearn.metrics import mean_squared_error

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


In [3]:
# 10-15 countries to start training
# Check whether the data file is updated and if we can train in predict.py

# Goal
Predict the daily number of cases for the given countries, regions and intervention plan.

This model is described in [From Prediction to Prescription: Evolutionary Optimization of Non-Pharmaceutical Interventions in the COVID-19 Pandemic](https://arxiv.org/abs/2005.13766) in Section 5 - Data-Driven Predictive Model.

Predictions from this model trained on the latest data are visible here: [How AI Makes Intervention Recommendations](https://evolution.ml/demos/npidashboard/). For more information see [Augmenting Human Decision Making
Optimizing COVID-19 Interventions](https://evolution.ml/esp/npi/).

## Intervention plan
An intervention plan consists in a list of containment and closure policies, as well as health system policies. See https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md for more details. The following policies are considered:

In [4]:
NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing',
               'H6_Facial Coverings']

## Example
For the given input, provide a model that can produce the expected output

### Given input
One row represents a country, region and day. For each row, the interventions that were in place on this day for this region and country are provided. Interventions can change from one day to the other for each country and region.

In [5]:
EXAMPLE_INPUT_FILE = "../../../validation/data/2020-09-30_historical_ip.csv"

In [6]:
prediction_input_df = pd.read_csv(EXAMPLE_INPUT_FILE,
                                  parse_dates=['Date'],
                                  dtype={"RegionName": str},
                                  encoding="ISO-8859-1")

In [7]:
prediction_input_df.head()

,CountryName,RegionName,Date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Expected output
For each row that was provided in input, i.e. for each country, region and day, the output should contain an additional `PredictedDailyNewCases` column with the predicted number of cases for that day, region and country. It is possible to leave `PredictedDailyNewCases` empty or NaN, or to remove the row, in case no predition is available.

In [8]:
EXAMPLE_OUTPUT_FILE = "../../../../2020-08-01_2020-08-04_predictions_example.csv"

In [9]:
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")

In [10]:
prediction_output_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
0,Aruba,NaN,2020-08-01,0.820071
1,Aruba,NaN,2020-08-02,0.872854
2,Aruba,NaN,2020-08-03,0.000000
3,Aruba,NaN,2020-08-04,0.000000
4,Afghanistan,NaN,2020-08-01,80.590128


### Evaluation
Predictions will be evaluated on a period of 4 weeks **after** submision against the actual daily change in confirmed cases reported by the [Oxford COVID-19 Government Response Tracker (OxCGRT)](https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker).

The latest data, including the latest confirmed cases ('ConfirmedCases') can be find here: https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv

In [11]:
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df = pd.read_csv(DATA_URL,
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [12]:
df.sample(3)

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
67402,Turkey,TUR,NaN,NaN,NAT_TOTAL,2020-08-13,3.0,1.0,2.0,1.0,...,48.15,48.15,63.10,63.10,61.61,61.61,57.29,57.29,87.5,87.5
69113,Uruguay,URY,NaN,NaN,NAT_TOTAL,2020-08-19,1.0,1.0,1.0,1.0,...,32.41,32.41,48.81,48.81,47.62,47.62,40.97,40.97,87.5,87.5
75626,United States,USA,Kentucky,US_KY,STATE_TOTAL,2020-09-22,2.0,0.0,1.0,1.0,...,62.96,62.96,72.62,72.62,67.26,67.26,70.14,70.14,50.0,50.0


### Daily change in confirmed cases
The daily change in confirmed cases can be computed like this:

In [13]:
df["DailyChangeConfirmedCases"] = df.groupby(["CountryName", "RegionName"]).ConfirmedCases.diff().fillna(0)

For instance, for country **United States**, region **California**, the latest available changes in confirmed cases are:

In [14]:
california_df = df[(df.CountryName == "United States") & (df.RegionName == "California")]

In [15]:
california_df[["CountryName", "RegionName", "Date", "ConfirmedCases", "DailyChangeConfirmedCases"]].tail(5)

,CountryName,RegionName,Date,ConfirmedCases,DailyChangeConfirmedCases
71264,United States,California,2020-12-02,1265182.0,19140.0
71265,United States,California,2020-12-03,1286550.0,21368.0
71266,United States,California,2020-12-04,1310307.0,23757.0
71267,United States,California,2020-12-05,1337941.0,27634.0
71268,United States,California,2020-12-06,NaN,0.0


# Training a model

## Copy the data locally

In [16]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
DATA_FILE = 'data/OxCGRT_latest.csv'

# Download the data set
data = requests.get(DATA_URL)

# Persist the data set locally in order to use it after submission to make predictions,
# as the sandbox won't have access to the internet anymore.
if not os.path.exists('data'):
    os.mkdir('data')
open(DATA_FILE, 'wb').write(data.content)

18259564

## Train

In [9]:
# Reload the module to get the latest changes
import xprize_predictor
from importlib import reload
reload(xprize_predictor)
from xprize_predictor import XPrizePredictor

In [38]:
predictor = XPrizePredictor(None, DATA_FILE)

In [39]:
%%time
predictor_model = predictor.train()

Creating numpy arrays for Keras for each country...
Numpy arrays created
Trial 0
143/143 [==============================] - 0s 2ms/step - loss: 0.0987
Train Loss: 0.10024679452180862
Val Loss: 0.09883466362953186
Test Loss: 0.09867030382156372
Done
CPU times: user 3min 47s, sys: 18.5 s, total: 4min 6s
Wall time: 2min 41s


In [40]:
if not os.path.exists('models'):
    os.mkdir('models')
predictor_model.save_weights("models/trained_model_weights_prediction_ratio_weights_mse.h5")

# Predicting using a trained model

## Load candidate model

In [21]:
model_weights_file = "models/trained_model_weights.h5"

In [22]:
predictor = XPrizePredictor(model_weights_file, DATA_FILE)

## Make prediction

In [23]:
NPIS_INPUT_FILE = "../../../validation/data/2020-09-30_historical_ip.csv"
start_date = "2020-08-01"
end_date = "2020-08-31"

In [24]:
%%time
preds_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

CPU times: user 1min 59s, sys: 2.69 s, total: 2min 1s
Wall time: 1min 53s


In [25]:
preds_date_df = preds_df[(preds_df['Date'] >= '2020-08-01') & (preds_df['Date'] <= '2020-08-31')]
date_df = df[(df['Date'] >= '2020-08-01') & (df['Date'] <= '2020-08-31')]

In [26]:
merged_df = pd.merge(date_df, preds_date_df, on=["Date", "CountryName", "RegionName"])
mse = mean_squared_error(merged_df['DailyChangeConfirmedCases'], merged_df['PredictedDailyNewCases'])
# merged_df['NewCasesDifference'] = merged_df['PredictedDailyNewCases'] - merged_df['DailyChangeConfirmedCases']
# merged_df[["CountryName", "RegionName", "Date", "DailyChangeConfirmedCases", "PredictedDailyNewCases", "NewCasesDifference"]].head()

Change test days to 60

In [27]:
model_60_weights_file = "models/trained_model_weights_60_test_days.h5"

In [28]:
predictor = XPrizePredictor(model_60_weights_file, DATA_FILE)

In [29]:
NPIS_INPUT_FILE = "../../../validation/data/2020-09-30_historical_ip.csv"
start_date = "2020-08-01"
end_date = "2020-08-31"

In [30]:
%%time
preds_60_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

CPU times: user 2min, sys: 2.74 s, total: 2min 3s
Wall time: 1min 54s


In [31]:
preds_date_60_df = preds_60_df[(preds_60_df['Date'] >= '2020-08-01') & (preds_60_df['Date'] <= '2020-08-31')]

In [32]:
merged_60_df = pd.merge(date_df, preds_date_60_df, on=["Date", "CountryName", "RegionName"])
mse_60 = mean_squared_error(merged_60_df['DailyChangeConfirmedCases'], merged_60_df['PredictedDailyNewCases'])
# merged_60_df['NewCasesDifference'] = merged_60_df['PredictedDailyNewCases'] - merged_60_df['DailyChangeConfirmedCases']
# merged_60_df[["CountryName", "RegionName", "Date", "DailyChangeConfirmedCases", "PredictedDailyNewCases", "NewCasesDifference"]].head()

In [33]:
(mse_60 - mse)/mse

-0.16697518960065255

In [34]:
# Use prediction ratio to sort countries + 60 days training
model_weights_pred_ratio_file = "models/trained_model_weights_prediction_ratio_weights.h5"
predictor = XPrizePredictor(model_weights_pred_ratio_file, DATA_FILE)

In [35]:
%%time
preds_ratio_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

CPU times: user 1min 59s, sys: 2.82 s, total: 2min 2s
Wall time: 1min 53s


In [36]:
preds_ratio_date_df = preds_ratio_df[(preds_ratio_df['Date'] >= '2020-08-01') & (preds_ratio_df['Date'] <= '2020-08-31')]
merged_preds_ratio_df = pd.merge(date_df, preds_ratio_date_df, on=["Date", "CountryName", "RegionName"])
mse_preds_ratio = mean_squared_error(merged_preds_ratio_df['DailyChangeConfirmedCases'], merged_preds_ratio_df['PredictedDailyNewCases'])

In [37]:
(mse_preds_ratio - mse)/mse

-0.49411831326455097

In [ ]:
# MSE

In [49]:
model_weights_pred_ratio_mse_file = "models/trained_model_weights_prediction_ratio_weights_mse.h5"
predictor = XPrizePredictor(model_weights_pred_ratio_mse_file, DATA_FILE)

In [50]:
%%time
preds_ratio_mse_df = predictor.predict(start_date, end_date, NPIS_INPUT_FILE)

CPU times: user 2min 7s, sys: 3.03 s, total: 2min 10s
Wall time: 2min 1s


In [51]:
preds_ratio_mse_date_df = preds_ratio_mse_df[(preds_ratio_mse_df['Date'] >= '2020-08-01') & (preds_ratio_mse_df['Date'] <= '2020-08-31')]
merged_preds_ratio_mse_df = pd.merge(date_df, preds_ratio_mse_date_df, on=["Date", "CountryName", "RegionName"])
mse_preds_ratio_mse = mean_squared_error(merged_preds_ratio_mse_df['DailyChangeConfirmedCases'], merged_preds_ratio_mse_df['PredictedDailyNewCases'])

In [52]:
(mse_preds_ratio_mse - mse)/mse

0.019430400288864506

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [21]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [22]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,0.0
Aruba,,2020-08-02,1.7698189576811538
Aruba,,2020-08-03,21.939474736193304
Aruba,,2020-08-04,25.106195049205034
Afghanistan,,2020-08-01,269.09904415313025
Afghanistan,,2020-08-02,155.55027796759626
Afghanistan,,2020-08-03,240.02544615184394
Afghanistan,,2020-08-04,200.2726904410946
Angola,,2020-08-01,15.565491775948328


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [24]:
# Check the pediction file is valid
import os
from predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [38]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Traceback (most recent call last):
  File "predict.py", line 68, in <module>
    predict(args.start_date, args.end_date, args.ip_file, args.output_file)
  File "predict.py", line 33, in predict
    predictor = XPrizePredictor(MODEL_WEIGHTS_FILE, DATA_FILE)
  File "/home/nitya/covid-xprize/covid_xprize/examples/predictors/lstm/xprize_predictor.py", line 81, in __init__
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
NameError: name 'physical_devices' is not defined


FileNotFoundError: [Errno 2] No such file or directory: 'predictions/val_4_days.csv'

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")